In [1]:
#Python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


#Guardar modelos
from scipy.sparse import csr_matrix
from scipy import sparse


#LDA
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from pprint import pprint

# Cargar datos

In [2]:
file_path_meta = "https://raw.githubusercontent.com/juankquintana/aprendizaje-no-supervisado/main/Proyecto/data/movies_metadata.csv"

metadata = pd.read_csv(file_path_meta)

metadata.head(1)

C:\Users\WD\AppData\Local\Temp\ipykernel_16708\834555082.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(file_path_meta)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [11]:
file_path_usuarios = "https://raw.githubusercontent.com/juankquintana/proyecto_recomendacion_peliculas/refs/heads/main/data/base_usuarios.csv"

df_usuarios = pd.read_csv(file_path_usuarios)

df_usuarios = df_usuarios.drop(df_usuarios.columns[0], axis=1)

df_usuarios.head(5)

,userId,movieId,rating
0,123687,163,4.0
1,221195,4973,4.0
2,134366,1643,4.0
3,20721,3034,3.5
4,225569,165,5.0


In [9]:
df_keywords_title = pd.read_pickle('ModelosNLP/df_keywords_title.pkl')
df_keywords_title.head(5)

,id,original_title,keyword_name,cleaned_keywords
0,2,Ariel,"[Drama, Crime, underdog, prison, factory worke...","[underdog, prison, factori worker, prison, hel..."
1,3,Varjoja paratiisissa,"[Drama, Comedy, salesclerk, helsinki, garbage,...","[salesclerk, helsinki, garbag, independ film]"
2,5,Four Rooms,"[Crime, Comedy, hotel, new year's eve, witch, ...","[hotel, new year eve, witch, bet, hotel room, ..."
3,6,Judgment Night,"[Action, Thriller, Crime, chicago, drug dealer...","[chicago, drug dealer, box match, escap, one n..."
4,11,Star Wars,"[Adventure, Action, Science Fiction, android, ...","[android, galaxi, hermit, death star, lightsab..."


## Cargar funciones de recomendacion

### Recomendador Coseno

In [ ]:
def recomendador_coseno(movie_id, cosine_sim, num_recomendaciones=5, df=df_keywords_title):
    # Crear una Serie que contiene como índice el 'id' y el valor correspondiente al índice de la película
    indices = pd.Series(df.index, index=df['id'])

    # Verificar si el movie_id está en los índices
    if movie_id not in indices.index:
        print(f'ID de película {movie_id} no encontrado.')
        return

    # Obtener el índice de la película a partir del movie_id
    idx = indices[movie_id]

    # Obtener el título de la película original
    original_title = df.loc[idx, 'original_title']
    print(f'Título de la película original: {original_title}')
    print(f'Índice de la película: {movie_id}')

    # Obtener la similitud de coseno para esa película en particular con todas las películas
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar la lista de tuplas por la similitud de coseno en orden descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las películas más similares ignorando el primero (que es la misma película)
    sim_scores = sim_scores[1:num_recomendaciones+1]

    # Obtener los índices y las similitudes
    movie_indices = [i[0] for i in sim_scores]
    sim_scores_values = [i[1] for i in sim_scores]

    # Verificar que los índices obtenidos están dentro del rango del DataFrame
    valid_indices = [idx for idx in movie_indices if idx in df.index]

    # Crear un DataFrame para mostrar los resultados
    result_df = pd.DataFrame({
        'Índice': valid_indices,
        'Título': df['original_title'].iloc[valid_indices].values,
        'Similitud Coseno': [sim_scores_values[movie_indices.index(idx)] for idx in valid_indices]
    })

    # Imprimir las películas recomendadas con sus índices y similitudes
    print('\nPelículas recomendadas:')
    

    # Retornar el DataFrame con los resultados
    return result_df

### Recomendador Correlacion

In [ ]:
from scipy.spatial.distance import correlation
import pandas as pd

def recomendador_correlacion(movie_id, cosine_sim, num_recomendaciones=5,df=df_keywords_title):
    # Crear una Serie que contiene como índice el 'id' y el valor correspondiente al índice de la película
    indices = pd.Series(df.index, index=df['id'])

    # Verificar si el movie_id está en los índices
    if movie_id not in indices.index:
        print(f'ID de película {movie_id} no encontrado.')
        return

    # Obtener el índice de la película a partir del movie_id
    idx = indices[movie_id]

    # Obtener el vector TF-IDF de la película objetivo
    target_vector = cosine_sim[idx]

    # Calcular la correlación entre el vector de la película objetivo y todos los vectores
    correlations = []
    for i in range(cosine_sim.shape[0]):
        if i != idx:
            vector = cosine_sim[i]
            # Correlación = 1 - distancia para convertir distancia en correlación
            corr = 1 - correlation(target_vector, vector)
            correlations.append((i, corr))

    # Ordenar la lista de tuplas por la correlación en orden descendente
    correlations = sorted(correlations, key=lambda x: x[1], reverse=True)

    # Obtener las películas más similares ignorando el primero (que es la misma película)
    correlations = correlations[1:num_recomendaciones+1]

    # Obtener los índices y las correlaciones
    movie_indices = [i[0] for i in correlations]
    correlation_values = [i[1] for i in correlations]

    # Verificar que los índices obtenidos están dentro del rango del DataFrame
    valid_indices = [idx for idx in movie_indices if idx in df.index]

    # Crear un DataFrame para mostrar los resultados
    result_df = pd.DataFrame({
        'Índice': valid_indices,
        'Título': df['original_title'].iloc[valid_indices].values,
        'Correlación': [correlation_values[movie_indices.index(idx)] for idx in valid_indices]
    })

    # Imprimir las películas recomendadas con sus índices y correlaciones
    print('\nPelículas recomendadas:')
    #print(result_df.to_string(index=False))

    # Retornar el DataFrame con los resultados
    return result_df

# Recomendador